# 🛡️ Qwen3 Security Agent: L4 Speed (vLLM + 4-bit)
**Hardware:** NVIDIA L4 (24GB) | **Mode:** 4-bit Quantization + vLLM

This pipeline is tuned for the L4. It creates space for **vLLM acceleration** by using 4-bit quantization,
solving the "minutes per step" slowness while fitting comfortably in 24GB VRAM.

In [1]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2


In [2]:
# @title Install FlashInfer v0.6.0rc1 (via uv)
import os

!uv pip uninstall flashinfer-python --system

# Install the specific RC version using uv
# --system flag is required in Colab to install into the global environment
!uv pip install flashinfer-python==0.6.0rc1 \
    --extra-index-url https://flashinfer.ai/whl/cu128/torch2.8 \
    --pre \
    --no-deps \
    --system
# Verify installation (Import check)
try:
    import flashinfer
    print(f"✅ FlashInfer version {flashinfer.__version__} installed successfully.")
except ImportError:
    print("⚠️ Installation complete. PLEASE RESTART RUNTIME (Runtime > Restart Session) to load the new kernels.")

Using Python 3.12.12 environment at: /usr
Uninstalled 1 package in 130ms
 - flashinfer-python==0.6.0rc1
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 76ms
Installed 1 package in 42ms
 + flashinfer-python==0.6.0rc1
⚠️ Installation complete. PLEASE RESTART RUNTIME (Runtime > Restart Session) to load the new kernels.


In [3]:
# @title 2. Configuration
import unsloth
from unsloth import FastLanguageModel, PatchFastRL
import torch
import os
import json
import gc
import re
from google.colab import drive, userdata
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from trl import GRPOConfig, GRPOTrainer
from datasets import load_dataset, Dataset

drive.mount('/content/drive')
try: os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
except: pass

class Config:
    base_path = "/content/drive/MyDrive/Qwen3_Security_Agent_Pipeline"
    sft_adapter = os.path.join(base_path, "sft_checkpoints", "final_sft_adapter")
    output_dir = os.path.join(base_path, "grpo_checkpoints")
    cache_dir = os.path.join(base_path, "cache")
    temp_merge_dir = "/content/merged_sft_model_temp"

    # Model Settings (L4 Balanced)
    base_model = "unsloth/Qwen3-4B-Instruct-2507"
    load_in_4bit = True   # <--- 4-bit enables vLLM space on 24GB

    # Training Settings
    batch_size = 4
    grad_accum = 1
    generations = 8       # 8 parallel rollouts (Standard GRPO)
    steps = 300

cfg = Config()
os.makedirs(cfg.output_dir, exist_ok=True)
os.makedirs(cfg.cache_dir, exist_ok=True)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 12-19 05:48:28 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# @title 3. Data Pipeline
import json
import random
from datasets import load_dataset

# STRICT PROMPT: Enforces concise <think> blocks via 1-shot example
SYSTEM_PROMPT = """You are an expert security analysis agent.
1. Analyze the finding and reason step-by-step inside <think> tags.
2. Output the final structured analysis as a valid JSON object inside <answer> tags.

Example Response:
<think>
The process 'nc' listening on port 4444 is suspicious. It matches the pattern of a reverse shell.
The risk is high due to potential unauthorized remote access.
</think>
<answer>
{
  "risk_score": 90,
  "severity": "high",
  "rationale": "Netcat listener detected on non-standard port indicating potential C2."
}
</answer>

Now analyze the following finding:"""

def prepare_data():
    import os
    # Reuse existing small dataset if available to save time
    if not os.path.exists("train_small.jsonl"):
        print("📉 Fetching dataset...")
        ds_stream = load_dataset("text", data_files={"train": "hf://datasets/jmazz/sys-scan_synthetic_dataset_v2/train.jsonl"}, split="train")
        lines = [x['text'] for x in ds_stream]
        random.seed(42)
        random.shuffle(lines)
        subset = lines[:int(len(lines) * 0.5)] # Keep 50%
        with open("train_small.jsonl", "w") as f:
            for line in subset:
                f.write(line + "\n" if not line.endswith("\n") else line)

    print("🔄 Loading dataset...")
    ds = load_dataset("text", data_files={"train": "train_small.jsonl"}, split="train")

    def parse_and_format(ex):
        try:
            x = json.loads(ex['text'])
            meta = x.get('metadata', {})
            if isinstance(meta, str):
                try: meta = json.loads(meta)
                except: meta = {}

            user = {
                "title": x.get("title"),
                "description": x.get("description"),
                "metadata": meta,
                "category": x.get("category", "general")
            }
            gt = {
                "risk_score": int(x.get("risk_score", 0)),
                "severity": x.get("severity", "info").lower(),
                "rationale": x.get("rationale", "")
            }
            return {
                "prompt": [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": json.dumps(user, indent=2)}
                ],
                "answer": json.dumps(gt)
            }
        except: return None

    return ds.map(parse_and_format, remove_columns=["text"]).filter(lambda x: x['prompt'] is not None)

train_dataset = prepare_data()
print(f"✅ Data Ready: {len(train_dataset)} examples")

🔄 Loading dataset...


Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1470 [00:00<?, ? examples/s]

✅ Data Ready: 1470 examples


In [5]:
# @title 4. Model Setup (4-bit + vLLM)
def setup_model():
    # 1. Merge SFT (CPU Offload)
    # We perform a high-precision merge first to burn in the SFT weights
    if not os.path.exists(cfg.temp_merge_dir):
        print("🔄 Merging SFT Adapters...")
        base = AutoModelForCausalLM.from_pretrained(cfg.base_model, device_map="cpu", dtype=torch.float16)
        tok = AutoTokenizer.from_pretrained(cfg.base_model)

        model = PeftModel.from_pretrained(base, cfg.sft_adapter)
        model = model.merge_and_unload()

        model.save_pretrained(cfg.temp_merge_dir)
        tok.save_pretrained(cfg.temp_merge_dir)
        del base, model
        gc.collect()

    # 2. Reload with vLLM + 4-bit
    print("🚀 Reloading in 4-bit with vLLM...")

    # Patch for vLLM support in Unsloth
    PatchFastRL("GRPO", FastLanguageModel)

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = cfg.temp_merge_dir,
        max_seq_length = 4096,
        load_in_4bit = True,     # <--- 4-bit Quantization
        fast_inference = True,   # <--- vLLM Enabled
        gpu_memory_utilization = 0.6, # 60% of 24GB for Weights + KV Cache
        cache_dir = cfg.cache_dir
    )

    model = FastLanguageModel.get_peft_model(
        model, r=64, lora_alpha=64,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        use_gradient_checkpointing="unsloth", random_state=3407
    )
    return model, tokenizer

model, tokenizer = setup_model()
print("✅ Model Ready (vLLM + 4-bit).")

🚀 Reloading in 4-bit with vLLM...
Unsloth: UnslothAlignPropTrainer is already patched.
Unsloth: UnslothBCOTrainer is already patched.
Unsloth: UnslothCPOTrainer is already patched.
Unsloth: UnslothDDPOTrainer is already patched.
Unsloth: UnslothDPOTrainer is already patched.
Unsloth: UnslothGKDTrainer is already patched.
Unsloth: UnslothGRPOTrainer is already patched.
Unsloth: UnslothIterativeSFTTrainer is already patched.
Unsloth: UnslothKTOTrainer is already patched.
Unsloth: UnslothNashMDTrainer is already patched.
Unsloth: UnslothOnlineDPOTrainer is already patched.
Unsloth: UnslothORPOTrainer is already patched.
Unsloth: UnslothPPOTrainer is already patched.
Unsloth: UnslothPRMTrainer is already patched.
Unsloth: UnslothRewardTrainer is already patched.
Unsloth: UnslothRLOOTrainer is already patched.
Unsloth: UnslothSFTTrainer is already patched.
Unsloth: UnslothXPOTrainer is already patched.
INFO 12-19 05:48:38 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
INFO 12-1

`torch_dtype` is deprecated! Use `dtype` instead!


WARNING 12-19 05:48:57 [__init__.py:2767] Casting torch.float16 to torch.bfloat16.
INFO 12-19 05:48:57 [__init__.py:1815] Using max model len 4096
INFO 12-19 05:48:59 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=4096.
WARNING 12-19 05:48:59 [lora.py:92] `lora_extra_vocab_size` is deprecated and will be removed in v0.12.0. Additional vocabulary support for LoRA adapters is being phased out.
WARNING 12-19 05:48:59 [_ipex_ops.py:16] Import error msg: No module named 'intel_extension_for_pytorch'
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'bfloat16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'fp4', 'bnb_4bit_use_double_quant': False, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': [], 'llm_int8_threshold': 6.0}
INFO 12-19 05:49:00 [core.py:76] Initializing a V1 LLM engine (v0.10.2) with config: model='/content/merg

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 12-19 05:49:04 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 12-19 05:49:05 [gpu_model_runner.py:2392] Model loading took 2.7898 GiB and 3.034783 seconds
INFO 12-19 05:49:20 [backends.py:539] Using cache directory: /root/.cache/vllm/torch_compile_cache/eb8b2232ad/rank_0_0/backbone for vLLM's torch.compile
INFO 12-19 05:49:20 [backends.py:550] Dynamo bytecode transform time: 14.52 s
INFO 12-19 05:49:28 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 5.525 s
INFO 12-19 05:49:32 [monitor.py:34] torch.compile takes 14.52 s in total
INFO 12-19 05:49:34 [gpu_worker.py:298] Available KV cache memory: 9.83 GiB
INFO 12-19 05:49:34 [kv_cache_utils.py:864] GPU KV cache size: 71,600 tokens
INFO 12-19 05:49:34 [kv_cache_utils.py:868] Maximum concurrency for 4,096 tokens per request: 17.48x
INFO 12-19 05:49:34 [vllm_utils.py:707] Unsloth: Running patched vLLM v1 `capture_model`.


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Qwen3ForCausalLM were not initialized from the model checkpoint at /content/merged_sft_model_temp and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Performing substitution for additional_keys=set()
Unsloth: Just some info: will skip parsing ['q_norm', 'norm2', 'cross_attn_post_attention_layernorm', 'cross_attn_input_layernorm', 'post_attention_layernorm', 'post_feedforward_layernorm', 'layer_norm2', 'norm', 'ffn_norm', 'post_layernorm', 'attention_norm', 'norm1', 'k_norm', 'pre_feedforward_layernorm', 'layer_norm1', 'input_layernorm']


Unsloth 2025.12.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


✅ Model Ready (vLLM + 4-bit).


In [6]:
# @title 5. Rewards
import re
import json

def extract_json(t):
    m = re.search(r"<answer>(.*?)</answer>", t, re.DOTALL)
    if m:
        try: return json.loads(m.group(1))
        except: return None
    return None

def json_format_reward(completions, **kwargs):
    rewards = []
    for c in completions:
        content = c[0]["content"]
        score = -1.0
        # Partial credit to guide convergence
        if "<think>" in content: score += 0.4
        if "</think>" in content: score += 0.4
        if "<answer>" in content: score += 0.4
        if extract_json(content): score = 2.0
        rewards.append(score)
    return rewards

def efficiency_reward(completions, **kwargs):
    rewards = []
    for c in completions:
        content = c[0]["content"]
        if "<answer>" in content:
            # Reward brevity only if format is roughly correct
            rewards.append(max(0.0, 1.0 - (len(content) / 2000.0)))
        else:
            rewards.append(0.0)
    return rewards

def risk_score_reward(completions, answer, **kwargs):
    rews = []
    for c, gt in zip(completions, answer):
        p, g = extract_json(c[0]["content"]), json.loads(gt)
        if not p: rews.append(0.0); continue
        try:
            diff = abs(float(p.get("risk_score", -1)) - float(g.get("risk_score", -1)))
            rews.append(1.0 - (diff/100.0) + (0.5 if diff<=5 else 0))
        except: rews.append(0.0)
    return rews

def severity_reward(completions, answer, **kwargs):
    rank = {"info":1, "low":2, "medium":3, "high":4, "critical":5}
    rews = []
    for c, gt in zip(completions, answer):
        p, g = extract_json(c[0]["content"]), json.loads(gt)
        if not p: rews.append(0.0); continue
        p_s, g_s = p.get("severity", "").lower(), g.get("severity", "").lower()
        if p_s == g_s: rews.append(1.0)
        else: rews.append(max(0, 1.0 - abs(rank.get(p_s,0)-rank.get(g_s,0))*0.25))
    return rews

In [ ]:
# @title 6. Run Training (vLLM Accelerated)
print("🚀 Starting GRPO (vLLM Mode)...")

args = GRPOConfig(
    output_dir=cfg.output_dir,
    learning_rate=5e-6,
    adam_beta1=0.9, adam_beta2=0.99,
    warmup_steps=30, logging_steps=1,
    per_device_train_batch_size=cfg.batch_size,
    gradient_accumulation_steps=cfg.grad_accum,
    num_generations=cfg.generations,
    max_prompt_length=1024,
    max_completion_length=2084,
    max_steps=cfg.steps,
    save_steps=50,
    report_to="none",

    # vLLM Settings for L4
    bf16=True, fp16=False,
    use_vllm=True,
    vllm_gpu_memory_utilization=0.35, # Reserve space for vLLM context
)

trainer = GRPOTrainer(
    model=model, processing_class=tokenizer,
    reward_funcs=[json_format_reward, efficiency_reward, risk_score_reward, severity_reward],
    args=args, train_dataset=train_dataset
)

trainer.train()
model.save_lora(os.path.join(cfg.output_dir, "final_pipeline_adapter"))
print("✅ Done! Saved to Drive.")

🚀 Starting GRPO (vLLM Mode)...
Unsloth: We now expect `per_device_train_batch_size` * `gradient_accumulation_steps` * `world_size` to be a multiple of `num_generations`.
We will change the batch size of 4 to the `num_generations` of 8


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,470 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 132,120,576 of 4,154,588,672 (3.18% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / json_format_reward / mean,rewards / json_format_reward / std,rewards / efficiency_reward / mean,rewards / efficiency_reward / std,rewards / risk_score_reward / mean,rewards / risk_score_reward / std,rewards / severity_reward / mean,rewards / severity_reward / std
